### Import required packages

In [1]:
from osgeo import gdal, osr, ogr
import matplotlib.pyplot as plt
import numpy as np
import os
import osmnx as ox
import rasterio
from rasterstats import zonal_stats
from rasterio.plot import show
from rasterio.enums import Resampling
import geopandas as gpd
from shapely.geometry import Polygon
import json
import math

### Set constants

In [79]:
def findGDALCoordinates(path):
    if not os.path.isfile(path):
        return []
    data = gdal.Open(path,0)
    if data is None:
        return []
    geoTransform = data.GetGeoTransform()
    minx = geoTransform[0]
    maxy = geoTransform[3]
    maxx = minx + geoTransform[1]*data.RasterXSize
    miny = maxy + geoTransform[5]*data.RasterYSize
    return [minx,maxy,maxx,miny]

In [89]:
from osgeo import gdal

input_dir = '../raster-images/basic.tif'
reference = gdal.Open(input_dir, 0)
rt = reference.GetGeoTransform()
xr = rt[1]
yr = -rt[5]
print(findGDALCoordinates("../raster-images/sentinel.tif"))
ds = gdal.Translate('../raster-images/testing.tif', input_dir, xRes=0.02, yRes=0.02, 
                    resampleAlg="bilinear", 
                    projWin = findGDALCoordinates("../raster-images/sentinel.tif"),
                    format='vrt');

ds.GetGeoTransform()



[47.42053671268654, -18.809913565707074, 47.62104068410202, -19.01041753712255]
